In [32]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding , Dropout
from keras.models import Model
import pandas as pd
import re
from sklearn.base import BaseEstimator, TransformerMixin


In [33]:
class TextCleaner(BaseEstimator, TransformerMixin):    
    def remove_mentions(self, text):        
        return re.sub(r'@\w+', '', text)
    
    def remove_urls(self, text):        
        return re.sub(r'http.?://[^\s]+[\s]?', '', text)
    
    def only_characters(self, text):
        return re.sub('[^a-zA-Z\s]', '', text)
    
    def remove_extra_spaces(self, text):
        text = re.sub("\s+", ' ', text)
        text = text.lstrip()
        return text.rstrip()
    
    def to_lower(self, text):
        return text.lower()
    
    def fix_words(self, text):
        text = re.sub(r'\bthx\b', 'thanks', text)
        text = re.sub(r'\bu\b', 'you', text)
        text = re.sub(r'\bhrs\b', 'hours', text)
        text = re.sub(r'\baa\b', 'a', text)
        text = re.sub(r'\bflightr\b', 'flight', text)
        text = re.sub(r'\bur\b', 'your', text)
        text = re.sub(r'\bhr\b', 'hour', text)
        text = re.sub(r'\bthru\b', 'through', text)
        text = re.sub(r'\br\b', 'are', text)
        text = re.sub(r'\bppl\b', 'people', text)
        text = re.sub(r'\btix\b', 'fix', text)
        text = re.sub(r'\bplz\b', 'please', text)
        text = re.sub(r'\bflightd\b', 'flighted', text)
        text = re.sub(r'\btmrw\b', 'tomorrow', text)
        text = re.sub(r'\bthx\b', 'thanks', text)
        text = re.sub(r'\bpls\b', 'please', text)
        text = re.sub(r'\bfyi\b', 'for your information', text)
        
        text = re.sub(r'\bheyyyy\b', 'hey', text)
        text = re.sub(r'\bguyyyys\b', 'guys', text)
        text = re.sub(r'\byall\b', 'you all', text)
        text = re.sub(r'\basap\b', 'as soon as possible', text)
        text = re.sub(r'\bbtw\b', 'by the way', text)
        text = re.sub(r'\bdm\b', 'direct message', text)
        text = re.sub(r'\bcudtomers\b', 'customers', text)
        text = re.sub(r'\bwtf\b', 'what the fuck', text)
        text = re.sub(r'\biphone\b', 'phone', text)
        text = re.sub(r'\bmins\b', 'minutes', text)
        text = re.sub(r'\btv\b', 'television', text)
        text = re.sub(r'\bokay\b', 'ok', text)
        text = re.sub(r'\bfeb\b', 'february', text)
        text = re.sub(r'\byr\b', 'year', text)
        text = re.sub(r'\bshes\b', 'she is', text)
        text = re.sub(r'\bnope\b', 'no', text)
        text = re.sub(r'\bhes\b', 'he is', text)
        text = re.sub(r'\btill\b', 'until', text)
        text = re.sub(r'\bomg\b', 'oh my god', text)
        text = re.sub(r'\btho\b', 'though', text)
        text = re.sub(r'\bnothappy\b', 'not happy', text)
        return re.sub(r'\bthankyou\b', 'thank you', text)
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):        
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.only_characters).apply(self.remove_extra_spaces).apply(self.to_lower).apply(self.fix_words)
        return clean_X

In [2]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'Task2')
MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 40000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [3]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [8]:
#dfXY  = pd.read_csv('labels_titles.csv')
df=pd.read_csv('Airline-Sentiment-2-w-AA.csv', encoding='ISO-8859-1',usecols=['text', 'airline_sentiment'])
#X_test, Y_test = read_csv('data/tesss.csv')
df.head()


,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [7]:
#df=pd.read_csv('Task2/orig.csv',encoding = "ISO-8859-1",header=None)

In [11]:
X=df.iloc[:,1]
y=df.iloc[:,0]

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['airline_sentiment'])

In [19]:
df['target'].head()

0    1
1    2
2    1
3    0
4    0
Name: target, dtype: int64

In [34]:
tc = TextCleaner()
df['clean_text'] = tc.transform(df['text'])

In [35]:
X=df['clean_text']
y=df['target']

X = np.array(dfXY.iloc[:,1]).astype(str)
y = np.array(dfXY.iloc[:,2]).astype(int)
print(X[:5])
print(y[:5])

itemindex1 = np.where(y<0)
for index in itemindex1:
    print(index)
    y[index]=np.random.randint(8)

In [36]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)                                #num_words basically tells the size of vocabulary.
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
print(sequences[2])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

[3, 179, 93, 743, 534, 3, 71, 1, 141, 136, 183]
Found 12523 unique tokens.


In [37]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (14640, 100)
Shape of label tensor: (14640, 3)


In [38]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.10, random_state=42)

In [39]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [40]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [41]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = Conv1D(128, 3, activation='relu')(embedded_sequences)
print(x.shape)
x = MaxPooling1D(3)(x)
print(x.shape)
x=Dropout(0.5)(x)
x = Conv1D(128, 3, activation='relu')(x)
print(x.shape)
x = MaxPooling1D(3)(x)
print(x.shape)
x = Conv1D(128, 5, activation='relu')(x)
print(x.shape)
x=Dropout(0.5)(x)
x = GlobalMaxPooling1D()(x)
print(x.shape)
x = Dense(128, activation='relu')(x)
print(x.shape)
preds = Dense(3, activation='softmax')(x)
print(preds.shape)

(?, 100, 100)
(?, 98, 128)
(?, 32, 128)
(?, 30, 128)
(?, 10, 128)
(?, 6, 128)
(?, 128)
(?, 128)
(?, 3)


In [43]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=30,
          validation_data=(x_val, y_val))

Train on 13176 samples, validate on 1464 samples
Epoch 1/30
13176/13176 [==============================] - 8s 612us/step - loss: 0.3877 - acc: 0.8434 - val_loss: 0.5221 - val_acc: 0.7951
Epoch 2/30
13176/13176 [==============================] - 7s 555us/step - loss: 0.3578 - acc: 0.8582 - val_loss: 0.5442 - val_acc: 0.7883
Epoch 3/30
13176/13176 [==============================] - 7s 561us/step - loss: 0.3309 - acc: 0.8659 - val_loss: 0.5642 - val_acc: 0.7801
Epoch 4/30
13176/13176 [==============================] - 7s 556us/step - loss: 0.3240 - acc: 0.8711 - val_loss: 0.5319 - val_acc: 0.7862
Epoch 5/30
13176/13176 [==============================] - 7s 554us/step - loss: 0.3072 - acc: 0.8749 - val_loss: 0.5385 - val_acc: 0.7876
Epoch 6/30
13176/13176 [==============================] - 7s 558us/step - loss: 0.2913 - acc: 0.8864 - val_loss: 0.5476 - val_acc: 0.7842
Epoch 7/30
13176/13176 [==============================] - 7s 560us/step - loss: 0.2706 - acc: 0.8906 - val_loss: 0.5594 - v

In [44]:
model.save('my_model.h5')

In [22]:
from keras.models import load_model
model = load_model('my_model.h5')